In [ ]:
import tszip
import sc2ts
import pandas as pd

In [ ]:
ts = tszip.load(snakemake.input[0])
ts

In [ ]:
ti = sc2ts.TreeInfo(ts, show_progress=False)
ti

In [ ]:
ti.plot_mutations_per_node_distribution();

In [ ]:
ti.plot_mutations_per_site_distribution();

In [ ]:
ti.plot_mutations_per_site(annotate_threshold=0.65);

In [ ]:
ti.plot_mutations_per_site(annotate_threshold=0.10, select=ti.mutations_is_reversion);

In [ ]:
ti.plot_deletion_overlaps(annotate_threshold=0.1);

In [ ]:
df = ti.deletions_summary()
df

In [ ]:
df.to_csv(snakemake.output[0]);

In [ ]:
df.length.value_counts()

In [ ]:
dfl = df[df.max_inheritors > 100].sort_values("max_inheritors")
dfl

In [ ]:
dfl.length.value_counts()

In [ ]:
df.sort_values("max_inheritors").tail(20)

# Examine the mutations

In [ ]:
dfm = pd.read_csv(snakemake.input[1])

In [ ]:
dfm[dfm.old > 500]

In [ ]:
dfm.old.hist();

In [ ]:
dfm.intersection.hist();

In [ ]:
dfm.new.hist();

In [ ]:
dfm.old.describe()

In [ ]:
dfm.new.describe()

In [ ]:
dfm.intersection.describe()